In [1]:
import numpy as np
from scipy.sparse import csr_matrix, dok_matrix
import scipy.sparse
from scipy.sparse import linalg as slinalg

adjacency_path = "./data/movielens20M.train.npz"
adjacency_train = scipy.sparse.load_npz(adjacency_path)

In [2]:
p2p_mat = adjacency_train.T.dot(adjacency_train)
p2p_mat = p2p_mat - scipy.sparse.diags(p2p_mat.diagonal())
p2p_mat.eliminate_zeros()

In [14]:
def get_pmi_matrix(adjacency, cds, neg, pos):
    sum_w = np.array(adjacency.sum(axis=1))[:, 0]
    sum_c = np.array(adjacency.sum(axis=0))[0, :]
    if cds != 1:
        sum_c = sum_c ** cds
    sum_total = sum_c.sum()
    sum_w = np.reciprocal(sum_w)
    sum_c = np.reciprocal(sum_c)

    pmi = multiply_by_rows(adjacency, sum_w)
    pmi = multiply_by_columns(pmi, sum_c)
    pmi = pmi * sum_total
    pmi.data = np.log(pmi.data) - np.log(neg) + np.log(pos)
    return pmi

def multiply_by_rows(matrix, row_coefs):
    normalizer = dok_matrix((len(row_coefs), len(row_coefs)))
    normalizer.setdiag(row_coefs)
    return normalizer.tocsr().dot(matrix)


def multiply_by_columns(matrix, col_coefs):
    normalizer = dok_matrix((len(col_coefs), len(col_coefs)))
    normalizer.setdiag(col_coefs)
    return matrix.dot(normalizer.tocsr())

cds = 0.75
neg = 1
pos = 100
pmi = get_pmi_matrix(p2p_mat, cds, neg, pos)

/Users/i.lobov/research/lib/python3.6/site-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in reciprocal
  import sys
/Users/i.lobov/research/lib/python3.6/site-packages/ipykernel_launcher.py:8: RuntimeWarning: divide by zero encountered in reciprocal
  


In [15]:
pmi.data.min()

1.3091650039935985

In [26]:
from sklearn.decomposition import TruncatedSVD

dim = 1000
# s, Vt = slinalg.eigsh(pmi, dim)
U, s, Vt = slinalg.svds(pmi, dim)
# U, s, Vt = slinalg.svds(adjacency_train, dim)
# svd = TruncatedSVD(n_components=dim, n_iter=1, random_state=0)
# U = svd.fit_transform(pmi)
# s = svd.singular_values_

# rng = np.random.RandomState(0)
# U = rng.normal(size=[pmi.shape[0], dim])
# U /= np.linalg.norm(U, axis=1, keepdims=True)
# U = pmi.T.dot(U)
# U, _ = np.linalg.qr(U)
# U /= np.linalg.norm(U, axis=1, keepdims=True)

# for _ in range(3):
#     U = pmi.dot(U)
#     U, _ = np.linalg.qr(U)
#     U /= np.linalg.norm(U, axis=1, keepdims=True)
    
#     U = pmi.T.dot(U)
#     U, _ = np.linalg.qr(U)
#     U /= np.linalg.norm(U, axis=1, keepdims=True)

In [27]:
#np.save("./data/movielens_users_pos=100", U)
#np.save("./data/movielens_movies_pos=100", Vt.T)
#np.save("./data/movielens_singular_vals_pos=100", s)

np.save("./data/movielens20M_users_svd", U)
np.save("./data/movielens20M_movies_svd", Vt.T)
np.save("./data/movielens20M_singular_vals_svd", s)

# np.save("./data/movielens20M_singular_vals_svd_pmi2", s)
# np.save("./data/movielens20M_movies_svd_pmi2", Vt)

# np.save("./data/movielens20M_singular_vals_rsvd1_pmi", s)
# np.save("./data/movielens20M_movies_rsvd1_pmi", U)

# np.save("./data/movielens20M_movies_pic_pmi", U)

In [13]:
# scipy.sparse.save_npz("./data/movielens.adjacency.train.npz", adjacency_train)
# scipy.sparse.save_npz("./data/movielens.adjacency.test.npz", adjacency_test)

In [13]:
np.save("./data/movielens20M_movies_pop", np.asarray(adjacency_train.sum(axis=0)).flatten())